In [1]:
from platform import python_version
print('Python version:', python_version())

Python version: 3.10.6


In [2]:
!nvidia-smi

Fri Mar 17 16:23:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 30%   31C    P8    N/A /  75W |    165MiB /  4096MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
import numpy as np
import tensorflow as tf
import pandas as pd
from matplotlib import pylab
import matplotlib
import matplotlib.gridspec as gridspec
%matplotlib inline
from nltk.translate.bleu_score import corpus_bleu
from sklearn.utils import shuffle
import word2vec
import nltk

## Dataset

[Dowload](https://nlp.stanford.edu/projects/nmt/):

* English vocabulary: [`vocab.50K.en`](https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/vocab.50K.en)

### Loading the Datasets and Building the Vocabulary

First, we build the vocabulary dictionaries for the source and target (English) language. 
The vocabularies are found in the file `vocab.50K.en`(English).

In [4]:
# Word string -> ID mapping
dictionary = dict()

vocabulary_size = len(dictionary)
with open('data/vocab.50K.en', encoding='utf-8') as f:
    for line in f:
        # disregard the new line aka `\n`
        dictionary[line[:-1]] = len(dictionary)
        
vocabulary_size = len(dictionary)
reverse_dictionary = dict(zip(dictionary.values(),dictionary.keys()))

print('Dictionary:', list(dictionary.items())[:10], end = '\n')
print('Reverse dictionary:', list(reverse_dictionary.items())[:10], end = '\n')
print('Vocabulary size: ', vocabulary_size, end = '\n')


Dictionary: [('<unk>', 0), ('<s>', 1), ('</s>', 2), ('the', 3), (',', 4), ('.', 5), ('of', 6), ('and', 7), ('to', 8), ('in', 9)]
Reverse dictionary: [(0, '<unk>'), (1, '<s>'), (2, '</s>'), (3, 'the'), (4, ','), (5, '.'), (6, 'of'), (7, 'and'), (8, 'to'), (9, 'in')]
Vocabulary size:  50000


### Loading data
Here we load the data from the dataset.csv file (generated in the other script)

In [5]:
dataset = pd.read_csv('data/dataset.csv')

### Data pre-processing
Transform to lower, remove the new line and the punctuation

In [6]:
wt = nltk.tokenize.WhitespaceTokenizer()

for column in dataset.columns:
    dataset[column] = dataset[column].str.lower() 
    dataset[column] = dataset[column].str.replace(',', ' ,')  \
                                     .str.replace('.',' .')   \
                                     .str.replace('?',' ?')   \
                                     .str.replace(')','')   \
                                     .str.replace('(','')   \
                                     .str.replace('"','')   \
                                     .str.replace('\n',' ')
    dataset[column] = dataset[column].apply(wt.tokenize)
dataset = shuffle(dataset)

/tmp/ipykernel_3620/518409875.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace('.',' .')   \
/tmp/ipykernel_3620/518409875.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace('?',' ?')   \
/tmp/ipykernel_3620/518409875.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace(')','')   \
/tmp/ipykernel_3620/518409875.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated

In [7]:
dataset.head()

,question,answer
341771,"[waiters/waitresses, ,, what, is, the, worst, ...","[i, was, on, a, first, date, and, we, were, co..."
32882,"[what's, legal, that, you, think, shouldn't, b...","[letting, children, go, unvaccinated]"
449456,"[who, is, a, person, in, history, that, had, a...","[vasilli, arkhipov, ., he, was, in, a, ussr, s..."
1090521,"[for, those, who, can't, seem, to, tell, their...","[him, ., and, why, he, loves, every, other, gi..."
594495,"[you're, an, evil, genius, who, wants, to, pun...","[i, would, engineer, cats, that, produce, urus..."


### Data analysis
Mean sentence length and standard deviation of sentence length

In [8]:
print('(Questions) Average sentence length: ', dataset['question'].str.len().mean())
print('(Questions) Standard deviation of sentence length: ', dataset['question'].str.len().std())

print('(Answers) Average sentence length: ', dataset['answer'].str.len().mean())
print('(Answers) Standard deviation of sentence length: ', dataset['answer'].str.len().std())

(Questions) Average sentence length:  17.101486059545056
(Questions) Standard deviation of sentence length:  9.12289135219408
(Answers) Average sentence length:  54.367627238247216
(Answers) Standard deviation of sentence length:  843.0636308326153


### Update the sentences to fixed length
Update all sentences with a fixed size, to process the sentences as batches.

In [9]:
max_sent_length = {'question' : 30, 'answer': 70}
def padding_sent(source):
    padded = []

    for tokens in dataset[source]: 
        # adding the start token
        tokens.insert(0, '<s>')  

        if len(tokens) >= max_sent_length[column]:
            tokens = tokens[:(max_sent_length[column] - 1)]
            tokens.append('</s>')

        if len(tokens) < max_sent_length[column]:
            tokens.extend(['</s>' for _ in range(max_sent_length[column] - len(tokens))])  

        padded.append(tokens)
    return padded

In [10]:
questions = padding_sent('question')
answers = padding_sent('answer')

### Create the reverse dataset

In [11]:
def create_reverse_dataset(source):
    reverse_tokens = []
    reverse_dataset = []
    for tokens in source: 
        for token in tokens: 
            if token not in dictionary.keys():
                reverse_tokens.append(dictionary['<unk>'])
            else:
                reverse_tokens.append(dictionary[token])
        reverse_dataset.append(reverse_tokens)
        reverse_tokens = []
    return reverse_dataset

train_inputs =  np.array(create_reverse_dataset(questions))
train_outputs =  np.array(create_reverse_dataset(answers))

### Word Embedding

In [ ]:
import word2vec

sentence_cursors = [0 for _ in range(train_inputs.shape[0])]

batch_size = 32
embedding_size = 64

word2vec.define_data_and_hyperparameters(
        train_inputs.shape[0], 
        max_sent_length['question'], 
        max_sent_length['answer'], 
        dictionary, 
        reverse_dictionary,  
        train_inputs, 
        train_outputs, 
        embedding_size,
        vocabulary_size)

word2vec.print_some_batches()
word2vec.define_word2vec_tensorflow(batch_size)
word2vec.run_word2vec(batch_size)
